<a id="1"></a> 
# <p style="padding:15px;background-color:#0F333D;margin:0;color:#D4FFCA;font-family:Verdana,sans-serif;font-size:9-0%;text-align:center;border-radius: 5px 5px;overflow:hidden;font-weight:900">Next Word Prediction: Marine Books</p>

In [5]:
!pip install pyforest

  Preparing metadata (setup.py) ... done
  Created wheel for pyforest: filename=pyforest-1.1.2-py2.py3-none-any.whl size=15900 sha256=83a083ccff1c74cf5f477d0c604652ea57c33fe14b81da50d34dad82912f84fc
  Stored in directory: /root/.cache/pip/wheels/c5/ca/73/5cdc3d087111bfbdef90be5457aa03c00c0e32241b2752445c
Successfully built pyforest


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import re
import joblib as jb
import tensorflow as tf
import plotly.graph_objects as go

In [6]:
from pyforest import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, InputLayer

In [7]:
with open('/kaggle/input/next-word-data/The Ocean World.txt', 'r', encoding='utf8') as dt:
    konten = [baris for baris in dt.readlines()]
    
konten[:5]

['The Project Gutenberg eBook of The Ocean World: Being a Description of the Sea and Its Living Inhabitants.\n',
 '    \n',
 'This ebook is for the use of anyone anywhere in the United States and\n',
 'most other parts of the world at no cost and with almost no restrictions\n',
 'whatsoever. You may copy it, give it away or re-use it under the terms\n']

<a id="1"></a> 
# <p style="padding:15px;background-color:#0F333D;margin:0;color:#D4FFCA;font-family:Verdana,sans-serif;font-size:9-0%;text-align:center;border-radius: 5px 5px;overflow:hidden;font-weight:900">Data Preprocessing</p>

In [8]:
# empty line/baris
clean = [baris.strip() for baris in konten if baris.strip()]
clean[:5]

['The Project Gutenberg eBook of The Ocean World: Being a Description of the Sea and Its Living Inhabitants.',
 'This ebook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this ebook or online']

In [9]:
# more like combine each sentence or line
clean = ' '. join(clean)
clean[:400]

'The Project Gutenberg eBook of The Ocean World: Being a Description of the Sea and Its Living Inhabitants. This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gute'

In [10]:
def cleaning(teks):
    # url, digits, special char, quotes, newline, etc with regex
    teks  = re.sub(r'http\S+|www\.\S+', '', teks)
    teks2 = re.sub(r'[^\w\s]|\r|\n|\d["\']', '', teks)
    teks2 = re.sub(r'[\s]+', ' ', teks2)
    
    return teks2

clean = cleaning(clean)
clean[:400]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'The Project Gutenberg eBook of The Ocean World Being a Description of the Sea and Its Living Inhabitants This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever You may copy it give it away or reuse it under the terms of the Project Gutenberg License included with this ebook or online at If you are no'

In [11]:
len(clean)

209965

### tokenization

In [12]:
token = Tokenizer()
token.fit_on_texts([clean])

In [13]:
data_sequence = token.texts_to_sequences([clean])[0]
print(data_sequence[:10])

[1, 1020, 1300, 701, 2, 1, 29, 123, 58, 6]


Example:

[1020] = 0

[2]

[0]

[4]

[123]

[431]

In [14]:
len(data_sequence)

36031

In [15]:
# eng vocab
vocab = len(token.word_index) + 1
vocab

5852

In [16]:
# cleaned text into sequence
# each row turn into vector representation in a sequence but for the first several word

# more length means more source
len_sequence = 3
sequence     = []

for seq in range(len_sequence, len(data_sequence)):
    
    # slicing to get next word as Target (y)
    
    # if i = 10, then [-36021:11]
    kata = data_sequence[seq - len_sequence: seq + 1]
    sequence.append(kata)

print(f'Length of the sequence: {len(sequence)}')
sequence = np.array(sequence)

Length of the sequence: 36028


<IPython.core.display.Javascript object>

In [17]:
# 20 first row
print(sequence[:10])

[[   1 1020 1300  701]
 [1020 1300  701    2]
 [1300  701    2    1]
 [ 701    2    1   29]
 [   2    1   29  123]
 [   1   29  123   58]
 [  29  123   58    6]
 [ 123   58    6  472]
 [  58    6  472    2]
 [   6  472    2    1]]


In [18]:
X = []
y = []

In [19]:
for sekuensial in sequence:
    X.append(sekuensial[0:len_sequence])
    y.append(sekuensial[len_sequence])

In [20]:
X = np.array(X)
y = np.array(y)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
# data and target for 3 row data
for a, b in zip(X[:3], y[:3]):
    print(f'{a} with target is [{b}]')

[   1 1020 1300] with target is [701]
[1020 1300  701] with target is [2]
[1300  701    2] with target is [1]


In [22]:
# target into categorical for model to abble clasify
y_kat = to_categorical(y, num_classes=vocab)

In [23]:
y_kat[13]

array([0., 0., 0., ..., 0., 0., 0.])

Now were able to conver any text into their vectir representation

<a id="1"></a> 
# <p style="padding:15px;background-color:#0F333D;margin:0;color:#D4FFCA;font-family:Verdana,sans-serif;font-size:9-0%;text-align:center;border-radius: 5px 5px;overflow:hidden;font-weight:900">Modeling</p>

In [24]:
# Long Short Term Memory

model6 = Sequential([
    InputLayer(shape=(len_sequence,)),
    Embedding(vocab, 100),
    LSTM(1000, return_sequences=True),
    LSTM(1000),
    Dense(1000, activation='relu'),
    Dense(vocab, activation='softmax')
])

In [25]:
model6.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 3, 100)         │       585,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 3, 1000)        │     4,404,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 1000)           │     8,004,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1000)           │     1,001,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5852)           │     5,857,852 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,852,052 (75.73 MB)

 Trainable params: 19,852,052 (75.73 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
model6.compile(optimizer='AdamW', loss=CategoricalCrossentropy())

In [27]:
checkpoint = ModelCheckpoint('model6.keras',
                             monitor='loss',
                             verbose=1,
                             save_best_only=True)

In [28]:
history6 = model6.fit(X,
                      y_kat,
                      epochs=45,
                      batch_size=64,
                      callbacks=[checkpoint])

Epoch 1/45
561/563 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 6.9738
Epoch 1: loss improved from inf to 6.70303, saving model to model6.keras
563/563 ━━━━━━━━━━━━━━━━━━━━ 17s 24ms/step - loss: 6.9723
Epoch 2/45
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 6.1498
Epoch 2: loss improved from 6.70303 to 6.11201, saving model to model6.keras
563/563 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - loss: 6.1497
Epoch 3/45
563/563 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 5.6897
Epoch 3: loss improved from 6.11201 to 5.66955, saving model to model6.keras
563/563 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - loss: 5.6897
Epoch 4/45
562/563 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 5.2459
Epoch 4: loss improved from 5.66955 to 5.25498, saving model to model6.keras
563/563 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - loss: 5.2459
Epoch 5/45
562/563 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 4.8697
Epoch 5: loss improved from 5.25498 to 4.89153, saving model to model6.keras
563/563 ━━━━━━━━━━━━━━━━━━━━ 14s 24ms/step - lo

In [32]:
loss_values = history6.history['loss']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(1, len(loss_values) + 1)),
    y=loss_values,
    mode='lines+markers',
    name='Training Loss',
    line=dict(color='firebrick', width=2),
    marker=dict(size=6)
))

# Add title and labels
fig.update_layout(
    title='Training Loss Over Epochs',
    xaxis_title='Epochs',
    yaxis_title='Loss',
    hovermode='x unified',
    template='plotly_dark'
)

fig.show()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
def clean_input(teks):
    # space remove
    teks_strip = teks.strip()
    clean_1 = cleaning(teks_strip)
    return clean_1

In [55]:
def prediksi_kata(teks):
    teks_split = teks.split(' ')[-3:]
    sekuensial = token.texts_to_sequences([teks_split])
    sekuensial = pad_sequences(sekuensial,
                               maxlen=len_sequence,
                               padding='pre')
    sekuensial = np.array(sekuensial)
    prediksi = np.argmax(model6.predict(sekuensial), axis=-1)
    
    pred_kata = ''
    for key, value in token.word_index.items():
        if value == prediksi:
            pred_kata = key
            break
    return pred_kata

In [56]:
teks1      = 'Marine is a'
no_word    = 34
clean_teks = clean_input(teks1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [57]:
clean_teks

'Marine is a'

In [58]:
for _ in range(no_word):
    pred_kata = prediksi_kata(clean_teks)
    clean_teks += ' ' + pred_kata

print(clean_teks)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Marine is a low sea water receives no fresh water into its bosom to maintain itself and flow off towards the atlantic ocean commences at the polar circle and reaches cape horn it is thus that the


In [59]:
# save
jb.dump(token, '/kaggle/working/tokenizer.sav')

['/kaggle/working/tokenizer.sav']

In [60]:
model6.save('/kaggle/working/model6.h5')